In [9]:
!pip install python-pptx --trusted-host pypi.org --trusted-host files.pythonhosted.org

     - 0 bytes ? 0:00:00
     - 2.5 kB ? 0:00:00
     - 0 bytes ? 0:00:00
     - 2.6 kB ? 0:00:00
   - 0 bytes ? 0:00:00
   - 320.4 kB 9.7 MB/s 0:00:00
   - 472.8 kB 9.8 MB/s 0:00:00
   - 0 bytes ? 0:00:00
   - 159.9 kB 10.0 MB/s 0:00:00


In [303]:
from pptx.dml.color import RGBColor
from pptx import Presentation
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
from pptx.util import Pt


In [305]:
# week1~3까지 데이터가 있다고 가정 (shorts 3개씩)
excel_file = "test_report_0103.xlsx"
week1 = pd.read_excel(excel_file, sheet_name="week1")
week2 = pd.read_excel(excel_file, sheet_name="week2")
week3 = pd.read_excel(excel_file, sheet_name="week3")

In [217]:
from pptx import Presentation

prs = Presentation("weekly_report.pptx")

# 레이아웃 확인
for i, layout in enumerate(prs.slide_layouts):
    print(f"Layout {i}: {layout.name}")

Layout 0: cover
Layout 1: contents
Layout 2: weekly_trend_insight
Layout 3: shorts_info
Layout 4: keywords


In [323]:
# 데이터프레임 생성 (주어진 데이터를 활용)
df = week1
df["check_date"] = pd.to_datetime(df["check_date"])
latest_data = df.loc[df.groupby("shorts_url")["check_date"].idxmax()]
sorted_data = latest_data.sort_values(by="shorts_view", ascending=False).reset_index(drop=True)

# 프레젠테이션 파일 로드
prs = Presentation("YT_report_ideation_test.pptx")

# 첫 번째 슬라이드에서 날짜 업데이트 함수
def update_date_on_slide(slide, today):
    for shape in slide.shapes:
        if shape.has_text_frame:
            if "2025-" in shape.text:
                shape.text = shape.text.replace("2025-", today)

# 첫 번째 슬라이드 날짜 업데이트
first_slide = prs.slides[0]
today_date = datetime.now().strftime("%Y-%m-%d")
update_date_on_slide(first_slide, today_date)

# 슬라이드 업데이트 함수
def update_slide_with_data(slide, views, likes, channel_name, title):
    for shape in slide.shapes:
        if shape.has_text_frame:
            # "조회수"를 업데이트
            if "조회수:" in shape.text:
                shape.text = shape.text.replace("조회수:", f"조회수: {views:,}")
            # "좋아요"를 업데이트
            elif "좋아요:" in shape.text:
                shape.text = shape.text.replace("좋아요:", f"좋아요: {likes:,}")
            elif "쇼츠제목" in shape.text:
                shape.text = shape.text.replace("쇼츠제목", title)
            # "채널명"을 업데이트
            elif "채널명:" in shape.text:
                shape.text = shape.text.replace("채널명:", f"채널명: {channel_name}")
            # 텍스트 크기를 12로 설정
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    run.font.size = Pt(10)

# 4~6 페이지에 데이터 적용
for i, slide_index in enumerate(range(3, 6)):  # 4번째, 5번째, 6번째 슬라이드
    if i < len(sorted_data):
        slide = prs.slides[slide_index]
        row = sorted_data.iloc[i]
        update_slide_with_data(
            slide,
            views=row["shorts_view"],
            likes=row["shorts_likes"],
            channel_name=row["channel_name"],
            title=row["shorts_title"]
        )

# 저장
prs.save("updated_weekly_report_key_data_with_date.pptx")

In [245]:
latest_data

,channel_name,shorts_url,shorts_title,shorts_description,shorts_thumbnail,shorts_view,shorts_likes,shorts_comments_num,shorts_comments_time,shorts_published_date,check_date,shorts_comments,is_paid
1,Matt Granger,https://www.youtube.com/watch?v=1uaDvx_UqPc,The $8200 waist level medium format camera,Retro - Modern - Classic. The CFV 100c back is...,https://i.ytimg.com/vi/1uaDvx_UqPc/maxres2.jpg...,2912,112,1,NaN,2024-12-19 23:50:00,2025-01-02 23:50:00,NaN,non_paid
3,Fine Art-Tips,https://www.youtube.com/watch?v=3wxe_Pk9q80,HOW TO DRAW WAVY HAIR - #shorts,"#shorts - If you enjoyed it, please like it an...",https://i.ytimg.com/vi/3wxe_Pk9q80/maxres2.jpg...,2885,138,5,NaN,2024-12-19 23:50:00,2025-01-02 23:50:00,NaN,non_paid
5,Knowing Better,https://www.youtube.com/watch?v=_V75fUlayts,Arguably Worse than Slavery | Company Towns | ...,When company coal towns didn't want to pay the...,https://i.ytimg.com/vi/_V75fUlayts/maxres2.jpg...,134646,8200,506,NaN,2024-12-19 23:50:00,2025-01-02 23:50:00,NaN,non_paid


In [327]:
import requests
from io import BytesIO
from pptx import Presentation
from pptx.util import Inches

# 프레젠테이션 파일 로드
prs = Presentation("YT_report_ideation_test.pptx")

# 이미지 추가 함수
def add_thumbnail_to_slide(slide, image_url, left, top, width, height):
    # URL에서 이미지 다운로드
    response = requests.get(image_url)
    if response.status_code == 200:
        image_stream = BytesIO(response.content)
        # 슬라이드에 이미지 추가
        slide.shapes.add_picture(image_stream, left, top, width, height)
    else:
        print(f"Failed to download image from {image_url}")

# 슬라이드 선택 및 이미지 추가
slide = prs.slides[0]  # 첫 번째 슬라이드
thumbnail_url = "https://i.ytimg.com/vi/1uaDvx_UqPc/maxres2.jpg"
add_thumbnail_to_slide(slide, thumbnail_url, left=Inches(1), top=Inches(1), width=Inches(2), height=Inches(1))

# 저장
prs.save("updated_presentation_with_thumbnail.pptx")


SSLError: HTTPSConnectionPool(host='i.ytimg.com', port=443): Max retries exceeded with url: /vi/1uaDvx_UqPc/maxres2.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))